# Toy model for estimation of connectivity in MOU process

This Python notebook estimates the connectivity matrix from the observed activity generated by a MOU process.

In [1]:
import os
import numpy as np
import scipy.linalg as spl
import scipy.stats as stt
from pyMOU.MOU_model import MOU
import pyMOU.tools as tools
import matplotlib.pyplot as pp

## Simulation

In [2]:
N = 10 # number of nodes
d = 0.3 # density of connectivity
# generate random matrix
C_orig = tools.make_rnd_connectivity(N, density=d, w_min=0.5/N/d, w_max=1.0/N/d)

# create MOU process
mou_orig = MOU(C_orig)

T = 500 # time in seconds
# simulate
ts_sim = mou_orig.simulate(T)

# plots
pp.figure()
pp.plot(range(T),ts_sim)
pp.xlabel('time')
pp.ylabel('activity')
pp.title('simulated MOU signals')

pp.show()

AttributeError: 'numpy.ndarray' object has no attribute 'minimum'

In [ ]:
Q_sim = np.tensordot(ts_sim,ts_sim,axes=(0,0)) / (T-1)

J = -np.eye(N) + C_orig
Sigma = np.eye(N)
Q_th = spl.solve_continuous_lyapunov(J,-Sigma)


# plots
pp.figure()
pp.imshow(Q_sim)
pp.colorbar()
pp.xlabel('source ROI')
pp.ylabel('target ROI')
pp.title('covariance matrix (functional connectivity)')

pp.figure()
pp.plot([0,Q_th.max()],[0,Q_th.max()],'--k')
pp.plot(Q_sim,Q_th,'.b')
pp.xlabel('simulated covariances')
pp.ylabel('theoretical covariances')

pp.show()

## Connectivity estimation

In [ ]:
# Lyapunov optimization
mou_est = MOU()
mou_est.fit(ts_sim, regul_C=1.)

C_est = mou_est.get_C()


# plots
pp.figure()
pp.imshow(C_orig,vmin=0)
pp.colorbar()
pp.xlabel('source ROI')
pp.ylabel('target ROI')
pp.title('original connectivity')

pp.figure()
pp.imshow(C_est,vmin=0)
pp.colorbar()
pp.xlabel('source ROI')
pp.ylabel('target ROI')
pp.title('estimated connectivity')

pp.figure()
pp.plot([0,C_orig.max()],[0,C_orig.max()],'--k')
pp.plot(C_est,C_orig,'xr')
pp.xlabel('estimated connectivity')
pp.ylabel('original connectivity')

pp.show()

In [ ]:
# moments method
mou_est2 = MOU()
mou_est2.fit(ts_sim,method='moments')

C_est2 = mou_est2.get_C()


# plots
pp.figure()
pp.imshow(C_orig,vmin=0)
pp.colorbar()
pp.xlabel('source ROI')
pp.ylabel('target ROI')
pp.title('original connectivity')

pp.figure()
pp.imshow(C_est2,vmin=0)
pp.colorbar()
pp.xlabel('source ROI')
pp.ylabel('target ROI')
pp.title('estimated connectivity')

pp.figure()
pp.plot([0,C_orig.max()],[0,C_orig.max()],'--k')
pp.plot(C_est2,C_orig,'xr')
pp.xlabel('estimated connectivity')
pp.ylabel('original connectivity')

pp.show()

In [ ]:
print('model fit for Lyapunov:',mou_est.d_fit['correlation'])
print('model fit for moments:',mou_est2.d_fit['correlation'])

print('C_orig fit for Lyapunov',stt.pearsonr(C_orig.flatten(),C_est.flatten()))
print('C_orig fit for moments',stt.pearsonr(C_orig.flatten(),C_est2.flatten()))
print('C_orig fit for moments + positive constraints',stt.pearsonr(C_orig.flatten(),np.maximum(C_est2,0).flatten()))

pp.figure()
pp.subplot(211)
pp.hist(C_est[C_orig>0],histtype='step',color='g')
pp.hist(C_est[C_orig==0],histtype='step',color='k')
pp.ylabel('distributions estimates')
pp.title('green = true; black = false')
pp.subplot(212)
pp.hist(C_est2[C_orig>0],histtype='step',color='g')
pp.hist(C_est2[C_orig==0],histtype='step',color='k')
pp.ylabel('distributions estimates')

pp.figure()
pp.subplot(211)
pp.hist(C_est[C_orig>0],histtype='step',cumulative=True,density=True,color='g')
pp.hist(C_est[C_orig==0],histtype='step',cumulative=True,density=True,color='k')
pp.ylabel('cumulative density')
pp.title('green = true; black = false')
pp.subplot(212)
pp.hist(C_est2[C_orig>0],histtype='step',cumulative=True,density=True,color='g')
pp.hist(C_est2[C_orig==0],histtype='step',cumulative=True,density=True,color='k')
pp.ylabel('cumulative density')

pp.show()